In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score,\
                                    StratifiedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.inspection import permutation_importance
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../../Data/master_df.csv')

In [3]:
just_pos = df.drop(columns=['text', 'is_conspiracy', 'X'])

y = df['is_conspiracy']
X = just_pos
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify = y,
                                                    random_state = 42)

In [4]:
folds = StratifiedKFold(shuffle = True, random_state=42)

In [13]:
params = {
    'learning_rate': [0.15],
    'n_estimators': [100],
    'max_depth': [2],
}

gs = GridSearchCV(GradientBoostingClassifier(), param_grid=params,
                  cv=folds, return_train_score=True)

gs.fit(X, y)

,params,mean_test_score,std_test_score,mean_train_score,std_train_score
0,"{'learning_rate': 0.15, 'max_depth': 2, 'n_est...",0.877333,0.009224,0.886223,0.001001
1,"{'learning_rate': 0.15, 'max_depth': 2, 'n_est...",0.877987,0.009634,0.888973,0.001410
2,"{'learning_rate': 0.15, 'max_depth': 3, 'n_est...",0.881852,0.007936,0.902604,0.002051
3,"{'learning_rate': 0.15, 'max_depth': 3, 'n_est...",0.882744,0.007915,0.905845,0.001352
4,"{'learning_rate': 0.2, 'max_depth': 2, 'n_esti...",0.879533,0.006189,0.890251,0.001813
5,"{'learning_rate': 0.2, 'max_depth': 2, 'n_esti...",0.879236,0.007462,0.893343,0.001619
6,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",0.882387,0.008151,0.908119,0.001373
7,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",0.882862,0.008728,0.912400,0.001302


In [16]:
results = pd.DataFrame(gs.cv_results_)
results[['params', 'mean_test_score', 'std_test_score',
         'mean_train_score', 'std_train_score']]

,params,mean_test_score,std_test_score,mean_train_score,std_train_score
0,"{'learning_rate': 0.15, 'max_depth': 2, 'n_estimators': 100}",0.877333,0.009224,0.886223,0.001001
1,"{'learning_rate': 0.15, 'max_depth': 2, 'n_estimators': 120}",0.877987,0.009634,0.888973,0.001410
2,"{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 100}",0.881852,0.007936,0.902604,0.002051
3,"{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 120}",0.882744,0.007915,0.905845,0.001352
4,"{'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 100}",0.879533,0.006189,0.890251,0.001813
5,"{'learning_rate': 0.2, 'max_depth': 2, 'n_estimators': 120}",0.879236,0.007462,0.893343,0.001619
6,"{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100}",0.882387,0.008151,0.908119,0.001373
7,"{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 120}",0.882862,0.008728,0.912400,0.001302


In [14]:
delta = 100

for test, train, i in zip(results.mean_test_score,
                          results.mean_train_score,
                          results.index):
    
    if (train - test) < delta:
        delta = (train - test)
        params = results.params[i]
        index = i
        
print(delta, params, index)

0.008890202829362925 {'learning_rate': 0.15, 'max_depth': 2, 'n_estimators': 100} 0


In [21]:
gbc = GradientBoostingClassifier(learning_rate=0.15, max_depth=2,
                                 n_estimators=100, random_state=42)
gbc.fit(X_train, y_train)
gbc.score(X_test, y_test), gbc.score(X_train, y_train)

(0.8763376932223543, 0.8863870609688417)

0.8732461355529132) (0.8798858320780147,

In [25]:
params = {
    'learning_rate': [0.9, 1.0, 1.1],
    'n_estimators': [40, 50, 60],
#     'max_depth': [2],
}

gs = GridSearchCV(AdaBoostClassifier(), param_grid=params,
                  cv=folds, return_train_score=True)

gs.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=AdaBoostClassifier(),
             param_grid={'learning_rate': [0.9, 1.0, 1.1],
                         'n_estimators': [40, 50, 60]},
             return_train_score=True)

In [26]:
results = pd.DataFrame(gs.cv_results_)
results[['params', 'mean_test_score', 'std_test_score',
         'mean_train_score', 'std_train_score']]

,params,mean_test_score,std_test_score,mean_train_score,std_train_score
0,"{'learning_rate': 0.9, 'n_estimators': 40}",0.866511,0.007532,0.869961,0.002650
1,"{'learning_rate': 0.9, 'n_estimators': 50}",0.867641,0.009275,0.872161,0.000861
2,"{'learning_rate': 0.9, 'n_estimators': 60}",0.868235,0.007215,0.872577,0.002097
3,"{'learning_rate': 1.0, 'n_estimators': 40}",0.864549,0.006425,0.869500,0.002837
4,"{'learning_rate': 1.0, 'n_estimators': 50}",0.865857,0.007630,0.871745,0.002231
5,"{'learning_rate': 1.0, 'n_estimators': 60}",0.867878,0.008743,0.873038,0.002046
6,"{'learning_rate': 1.1, 'n_estimators': 40}",0.867106,0.006297,0.870526,0.002886
7,"{'learning_rate': 1.1, 'n_estimators': 50}",0.863835,0.007945,0.871507,0.001976
8,"{'learning_rate': 1.1, 'n_estimators': 60}",0.867106,0.007353,0.873528,0.002469
